In [22]:

import sys, os
cdir = %pwd
sys.path.append(os.path.join(cdir, '..', 'python'))
sys.path.append(os.path.join(cdir, 'kunst', 'python'))

In [23]:
from hue_show import *

In [52]:
!curl https://www.meethue.com/api/nupnp

[{"id":"001788fffe273a83","internalipaddress":"192.168.1.21"},{"id":"001788fffe0aa167","internalipaddress":"192.168.1.113"}]

In [53]:
from time import sleep
from phue import Bridge
import random
from numpy import zeros, ones
import numpy as np
import matplotlib.pyplot as plt
from tqdm import *
import requests as req
import pandas as pd
from rgb_cie import Converter
import yaml

In [54]:
#bridges = req.get('https://www.meethue.com/api/nupnp').json()

In [55]:
with open('bridges.yml', 'r') as f:
    bridges = yaml.load(f.read())

In [56]:
from copy import copy

In [57]:
bri = []
for k, br in bridges.items():
    br['ip'] = '{IP}:{port}'.format(**br)
    b = Bridge(ip='{IP}:{port}'.format(**br), username=br['username'])
    b.connect
    bri.append(b)


In [58]:
bri

In [59]:
ln = 1
speed = 20 # m/s
distance = 7 #m between each lamp
time = 7 / (speed * 1E3 / (60*60))

def light(br, name, r,g,b):
    l = br.lights_by_name[name]
    rgbl(l, 1.0,0.0,0.0, brightness=254)

#normal()    
for n in range(9):
    try:
        name = "P%d"%((ln+n)%(19)+1)
        l = b.lights_by_name[name]
        rgbl(l, 1.0,0.0,0.0, brightness=254)
        sleep(time)
        rgbl(l, 1.,1.,1., brightness=254)
        sleep(0.1)
    except Exception as e:
        print(e)


'P2'
'P3'
'P4'
'P5'
'P6'
'P7'
'P8'
'P9'
'P10'


### Search for lights

In [60]:
import json
import time

In [61]:
fbri = lambda i, str_: 'http://{ip}/api/{username}/{str_}'.format(str_=str_, **bridges[i])
get = lambda i, str_:req.get(fbri(i, str_)).json()
post = lambda i, str_, data:req.post(fbri(i, str_), json.dumps(data)).json()
put = lambda i, str_,  data:req.put(fbri(i, str_), json.dumps(data)).json()

Search for new lights

In [63]:
for i in bridges.keys():
    post(i,'lights',{})

In [66]:
for i in bridges.keys():
    print(i)
    print(get(i,'lights/new'))

hue_in
{'lastscan': 'active', '61': {'name': 'Hue color lamp 20'}}
home1
{'lastscan': 'active'}
home2
{'lastscan': 'active'}


In [69]:
for i in bridges.keys():
    print('bridge: %s, now there are %d lights'%(i, len(get(i,'lights'))))

bridge: hue_in, now there are 37 lights
bridge: home1, now there are 40 lights
bridge: home2, now there are 29 lights


In [70]:
#df_lamps = []

def update_state():
    lamps = {}
    for i in bridges.keys():
        #print('Bridge-%d'%(i))
        for k,v in get(i,'lights').items():
            v.update(v['state'])
            v['bridge'] = i
            v['relid'] = k
            #df_lamps.append(v)
            #if 'Hue color' in v['name']:
            #print(v['name'], v['state']['reachable'], v['uniqueid'])
            idl = v['uniqueid']
            if idl in lamps:
                lamps[idl][i] = v
            else:
                lamps[idl] = {i:v}
        time.sleep(0.1)
    return lamps

In [71]:
update_state()

{'00:17:88:01:00:b1:f1:68-0b': {'home2': {'alert': 'none',
   'bri': 254,
   'bridge': 'home2',
   'colormode': 'ct',
   'ct': 369,
   'effect': 'none',
   'hue': 14910,
   'manufacturername': 'Philips',
   'modelid': 'LCT001',
   'name': 'Reol',
   'on': True,
   'reachable': False,
   'relid': '1',
   'sat': 144,
   'state': {'alert': 'none',
    'bri': 254,
    'colormode': 'ct',
    'ct': 369,
    'effect': 'none',
    'hue': 14910,
    'on': True,
    'reachable': False,
    'sat': 144,
    'xy': [0.4596, 0.4105]},
   'swversion': '5.23.1.13452',
   'type': 'Extended color light',
   'uniqueid': '00:17:88:01:00:b1:f1:68-0b',
   'xy': [0.4596, 0.4105]}},
 '00:17:88:01:00:b1:f8:dc-0b': {'home1': {'alert': 'none',
   'bri': 254,
   'bridge': 'home1',
   'colormode': 'ct',
   'ct': 369,
   'effect': 'none',
   'hue': 14910,
   'manufacturername': 'Philips',
   'modelid': 'LCT001',
   'name': 'Hue color lamp 17',
   'on': True,
   'reachable': True,
   'relid': '42',
   'sat': 144,
   

In [73]:
lamps = update_state()
for k,v in lamps.items():
    print(k,'-----')
    for l,m in v.items():
        print(l, m['state']['reachable'], m['name'], m['bridge'], m['relid'])
        if m['reachable']:
            put(m['bridge'], 'lights/%s/state'%(m['relid']), {"on":True, "sat":1, "bri":254,"hue":5000})
            time.sleep(0.5)
            print(get(m['bridge'], 'lights/%s'%(m['relid'])))
            time.sleep(0.5)
            

00:17:88:01:00:f5:7c:ae-0b -----
hue_in False Hue color lamp 2 hue_in 43
home1 False P13 home1 7
home2 False Hue Lamp 13 home2 13
00:17:88:01:00:cf:a5:dc-0b -----
home2 False Hue lightstrip 1 home2 28
home1 False Hue lightstrip 1 home1 39
00:17:88:01:00:ee:c4:2e-0b -----
hue_in False Hue color lamp 12 hue_in 53
home1 False P8 home1 8
home2 False Hue Lamp 27 home2 27
00:17:88:01:00:ee:c6:91-0b -----
home1 False P7 home1 17
00:17:88:01:00:f5:8f:f2-0b -----
hue_in False c1 hue_in 19
home1 False Hue color lamp 10 home1 34
00:17:88:01:00:bf:2f:6d-0b -----
hue_in True Hue color lamp 17 hue_in 58
{'name': 'Hue color lamp 17', 'manufacturername': 'Philips', 'uniqueid': '00:17:88:01:00:bf:2f:6d-0b', 'swversion': '5.23.1.13452', 'state': {'bri': 254, 'colormode': 'hs', 'alert': 'none', 'sat': 1, 'reachable': True, 'on': True, 'effect': 'none', 'hue': 5000, 'ct': 369, 'xy': [0.4596, 0.4105]}, 'modelid': 'LCT001', 'type': 'Extended color light'}
home1 True Hue color lamp 19 home1 44
{'name': 'Hue 

In [111]:
class SvalinMetaBridge(object):
    default_bridges = []
    
    all_lights_uid = ["DF1430", "7AC331", "B055E9", "1913F7", "2D2DA6", "0F9E4E", "DDE088", 
                      "209C5A", "E2B400", "6878AA", "34DE81", "62F1E6", "D9953F", "2A75A7", 
                      "5D3F46", "132A13", "81D5FE", "CF5C37"]
    
    def __init__(self, bridges={}):
        if len(bridges) > 0:
            self.bridges = bridges
        else:
            with open('bridges.yml', 'r') as f:
                self.bridges = yaml.load(f.read())
            
        self.lamps = {}
        self.bridge_state = {k:{} for k in self.bridges.keys()}
        
    def _fbri(self, i, str_): 
        return 'http://{IP}:{port}/api/{username}/{str_}'.format(str_=str_, **self.bridges[i])
    
    def search_all(self):
        for i in bridges.keys():
            for j in self.all_lights_uid:
                post(i,'lights', {"deviceid":[j]})
                time.sleep(0.5)

    def _get(self, i, str_):
        return req.get(self._fbri(i, str_)).json()

    def _post(self, i, str_, data):
        return req.post(self._fbri(i, str_), json.dumps(data)).json()
    
    def _put(self, i, str_,  data):
        return req.put(self._fbri(i, str_), json.dumps(data)).json()
        
    def get_bridge_state(self, i):
        self.bridge_state[i] = self._get(i,'lights')
        return self.bridge_state[i]
    
    def all_ellipse(self):
        return (('P%d'%(i+1), self.find_uid('P%d'%(i+1))) for i in range(18))
        
    def update_state(self):
        for i in bridges.keys():
            for k,v in self.get_bridge_state(i).items():
                v.update(v['state'])
                v['bridge'] = i
                v['relid'] = k
                idl = v['uniqueid']
                if idl in self.lamps:
                    self.lamps[idl][i] = v
                else:
                    self.lamps[idl] = {i:v}
            time.sleep(0.01)
            
    def equal_state(self, uid, target, count=10):
        self.update_state()
        try:
            current=self.get_state(uid)
        except Exception as e:
            if 'light unreachable' in str(e):
                if count>0:                
                    time.sleep(0.5)
                    self.equal_state(uid, target, count=count-1)
                else:
                    raise e
        for k,v in target.items():
            if not target[k] == current[k]:
                return False
        return True

    def set_uid_state(self, uid, state, count=10, check=True):
        for k, v in self.lamps[uid].items():
            #if v['reachable']:
            self._put(k, 'lights/%s/state'%(v['relid']), state)
            #time.sleep(0.1)
        if check:
            if not self.equal_state(uid, state, count):
                if count>0:
                    time.sleep(1.0)
                    self.set_uid_state(uid, state, count=count-1)
                else:
                    raise Exception('Cant set state', uid, state, self.get_state(uid))
                
    def set_state(self, name, state, count=10, check=True):
        self.set_uid_state(self.find_uid(name), state, count, check)
        
    def set_RBG(self, name, R, G, B, count=10, check=True):
        self.set_state(name, {'xy': c.rgbToCIE1931(R, G, B), 'on': True, 'bri':254}, count, check)

    def get_state(self, uid):
        for k, v in self.lamps[uid].items():
            if v['reachable']:
                return v
        raise Exception('light unreachable', uid)
    
    def query_state(self, uid):
        for k, v in self.lamps[uid].items():
            if v['reachable']:
                v.update(self._get(k, 'lights/%s'%(v['relid'])))
                return v        
        
    def find_uid(self, name):
        for k,v in self.lamps.items():
            for b, w in v.items():
                if w['name'] == name:
                    return k
        raise Exception('cant find lamp named', name)
        
    def __getitem__(self, key):
        return pd.DataFrame(self.invert(self.lamps[self.find_uid(key)]))
    
    def __getattr__(self, key):
        return pd.DataFrame(self.invert(self.lamps[self.find_uid(key)]))    
        
    def invert(self, lamp):
        new_dic = {}
        for k,v in lamp.items():
            for l, w in v.items():
                new_dic[l] = [None for b in self.bridges]
        for b in range(len(self.bridges)):
            for w in new_dic.keys():
                if b in lamp:
                    new_dic[w][b]=lamp[b][w]
        return new_dic
    
    def get_attribute(self, key):
        for k,v in self.lamps.items():
            print(k,':')
            print(', '.join(['%s:%s'%(k2, str(w[key])) for k2, w in v.items()]))
                
        #return pd.DataFrame([(k, k2, w[key]) for k2, w in v.items() for k,v in self.lamps.items()], columns=['uid', 'bridge', key]
        #                   ).pivot(index='uid', columns='bridge', values=key)

In [112]:
s = SvalinMetaBridge()

In [113]:
s.update_state()

In [114]:
s.get_bridge_state('hue_in')

{'1': {'manufacturername': 'Philips',
  'modelid': 'LCT001',
  'name': 'sofa1',
  'state': {'alert': 'none',
   'bri': 254,
   'colormode': 'xy',
   'ct': 346,
   'effect': 'none',
   'hue': 14956,
   'on': True,
   'reachable': False,
   'sat': 140,
   'xy': [0.445, 0.4067]},
  'swversion': '5.50.1.19085',
  'type': 'Extended color light',
  'uniqueid': '00:17:88:01:00:f5:91:ca-0b'},
 '10': {'manufacturername': 'Philips',
  'modelid': 'LCT001',
  'name': 'b2',
  'state': {'alert': 'none',
   'bri': 254,
   'colormode': 'xy',
   'ct': 346,
   'effect': 'none',
   'hue': 6885,
   'on': True,
   'reachable': False,
   'sat': 121,
   'xy': [0.445, 0.4067]},
  'swversion': '5.50.1.19085',
  'type': 'Extended color light',
  'uniqueid': '00:17:88:01:00:f5:91:b0-0b'},
 '12': {'manufacturername': 'Philips',
  'modelid': 'LCT001',
  'name': 'c3',
  'state': {'alert': 'none',
   'bri': 254,
   'colormode': 'xy',
   'ct': 346,
   'effect': 'none',
   'hue': 7114,
   'on': True,
   'reachable': F

In [91]:
s.query_state('00:17:88:01:00:ec:91:5b-0b')

{'alert': 'none',
 'bri': 254,
 'bridge': 'hue_in',
 'colormode': 'hs',
 'ct': 251,
 'effect': 'none',
 'hue': 5000,
 'manufacturername': 'Philips',
 'modelid': 'LCT001',
 'name': 'Hue color lamp 19',
 'on': True,
 'reachable': True,
 'relid': '60',
 'sat': 1,
 'state': {'alert': 'none',
  'bri': 254,
  'colormode': 'hs',
  'ct': 251,
  'effect': 'none',
  'hue': 5000,
  'on': True,
  'reachable': True,
  'sat': 1,
  'xy': [0.3814, 0.3767]},
 'swversion': '5.23.1.13452',
 'type': 'Extended color light',
 'uniqueid': '00:17:88:01:00:ec:91:5b-0b',
 'xy': [0.3814, 0.3767]}

In [93]:
s.lamps[uid]

{'home1': {'alert': 'none',
  'bri': 0,
  'bridge': 'home1',
  'colormode': 'hs',
  'ct': 0,
  'effect': 'none',
  'hue': 0,
  'manufacturername': 'Philips',
  'modelid': 'LCT001',
  'name': 'P1',
  'on': False,
  'reachable': False,
  'relid': '22',
  'sat': 0,
  'state': {'alert': 'none',
   'bri': 0,
   'colormode': 'hs',
   'ct': 0,
   'effect': 'none',
   'hue': 0,
   'on': False,
   'reachable': False,
   'sat': 0,
   'xy': [0.0, 0.0]},
  'swversion': '5.23.1.13187',
  'type': 'Extended color light',
  'uniqueid': '00:17:88:01:00:d9:c9:cf-0b',
  'xy': [0.0, 0.0]},
 'home2': {'alert': 'none',
  'bri': 240,
  'bridge': 'home2',
  'colormode': 'ct',
  'ct': 346,
  'effect': 'none',
  'hue': 0,
  'manufacturername': 'Philips',
  'modelid': 'LCT001',
  'name': 'Hue Lamp 9',
  'on': True,
  'reachable': False,
  'relid': '9',
  'sat': 0,
  'state': {'alert': 'none',
   'bri': 240,
   'colormode': 'ct',
   'ct': 346,
   'effect': 'none',
   'hue': 0,
   'on': True,
   'reachable': False

In [109]:
s.update_state()
for name, uid in s.all_ellipse():
    print(uid, ', '.join([l['name'] for l in s.lamps[uid].values()]), 
          s.invert(s.lamps[uid])['reachable'], 
          any(s.invert(s.lamps[uid])['reachable']))
    #s.set_RBG(name, 1.0, 1.0, 1.0, check=False)

00:17:88:01:00:d9:c9:cf-0b Hue color lamp 5, P1, Hue Lamp 9 [None, None, None] False
00:17:88:01:00:f5:8e:ff-0b Hue color lamp 7, P2, Hue Lamp 7 [None, None, None] False
00:17:88:01:00:d9:cb:67-0b Hue color lamp 13, P3, Hue Lamp 10 [None, None, None] False
00:17:88:01:00:ec:91:5b-0b Hue color lamp 19, P4, Hue Lamp 20 [None, None, None] False
00:17:88:01:00:ee:c7:5f-0b Hue color lamp 14, P5, Hue Lamp 14 [None, None, None] False
00:17:88:01:00:ec:91:a2-0b P6 [None, None, None] False
00:17:88:01:00:ee:c6:91-0b P7 [None, None, None] False
00:17:88:01:00:ee:c4:2e-0b Hue color lamp 12, P8, Hue Lamp 27 [None, None, None] False
00:17:88:01:00:ec:92:7e-0b Hue color lamp 9, P9, Hue Lamp 18 [None, None, None] False
00:17:88:01:00:d9:cb:35-0b Hue color lamp 4, P10, Hue Lamp 19 [None, None, None] False
00:17:88:01:00:f5:92:2e-0b Hue color lamp 1, P11, Hue Lamp 16 [None, None, None] False
00:17:88:01:00:f5:71:01-0b Hue color lamp 6, P12, Hue Lamp 17 [None, None, None] False
00:17:88:01:00:f5:7c:ae-0

In [102]:
s.P1.reachable

0    None
1    None
2    None
Name: reachable, dtype: object

In [103]:
s.P4

,alert,bri,bridge,colormode,ct,effect,hue,manufacturername,modelid,name,on,reachable,relid,sat,state,swversion,type,uniqueid,xy
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [105]:
s.set_RBG('P4', 1., 1., 1., check=False)

In [106]:
s.P8

,alert,bri,bridge,colormode,ct,effect,hue,manufacturername,modelid,name,on,reachable,relid,sat,state,swversion,type,uniqueid,xy
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [417]:
s._get(0, 'lights/23')

{'manufacturername': 'Philips',
 'modelid': 'LCT001',
 'name': 'P16',
 'state': {'alert': 'none',
  'bri': 254,
  'colormode': 'hs',
  'ct': 379,
  'effect': 'none',
  'hue': 14910,
  'on': True,
  'reachable': True,
  'sat': 254,
  'xy': [0.4656, 0.413]},
 'swversion': '5.23.1.13187',
 'type': 'Extended color light',
 'uniqueid': '00:17:88:01:00:dc:ea:dc-0b'}

In [421]:
s.set_state('P2', {'on':True, 'sat':254})

In [117]:
s.Stairs

,alert,bri,bridge,colormode,ct,effect,hue,manufacturername,modelid,name,on,reachable,relid,sat,state,swversion,type,uniqueid,xy
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [108]:
s.update_state()
s.P1.reachable

0    None
1    None
2    None
Name: reachable, dtype: object

In [362]:
anew_dic).T

SyntaxError: invalid syntax (<ipython-input-362-ecbba7370044>, line 1)

In [278]:
s.set_state('00:17:88:01:00:f5:71:01-0b',  {"on":True, "sat":254, "bri":254,"hue":5000})

Exception: ('light unreachable', '00:17:88:01:00:f5:71:01-0b')

In [115]:
s.get_attribute('name')

00:17:88:01:00:f5:7c:ae-0b :
hue_in:Hue color lamp 2, home1:P13, home2:Hue Lamp 13
00:17:88:01:00:cf:a5:dc-0b :
home2:Hue lightstrip 1, home1:Hue lightstrip 1
00:17:88:01:00:ee:c4:2e-0b :
hue_in:Hue color lamp 12, home1:P8, home2:Hue Lamp 27
00:17:88:01:00:ee:c6:91-0b :
home1:P7
00:17:88:01:00:f5:8f:f2-0b :
hue_in:c1, home1:Hue color lamp 10
00:17:88:01:00:bf:2f:6d-0b :
hue_in:Hue color lamp 17, home1:Hue color lamp 19, home2:Stairs
00:17:88:01:00:ba:99:e7-0b :
home2:Sofa, home1:Hue color lamp 18
00:17:88:01:00:f5:4a:93-0b :
hue_in:b3, home1:Hue color lamp 7, home2:Hue Lamp 25
00:17:88:01:00:d9:bd:12-0b :
hue_in:Hue color lamp 11, home1:P17, home2:Hue Lamp 3
00:17:88:01:00:dc:f6:9b-0b :
hue_in:a4, home1:Hue color lamp 5, home2:Hue Lamp 26
00:17:88:01:00:f5:91:ca-0b :
hue_in:sofa1
00:17:88:01:00:f5:6e:d7-0b :
hue_in:k2, home1:Hue color lamp 1, home2:Hue color lamp 3
00:17:88:01:00:dc:fe:12-0b :
hue_in:a2, home1:Hue color lamp 4, home2:Hue Lamp 21
00:17:88:01:00:f5:91:b0-0b :
hue_in:b2, 

In [218]:
all_lights_uid = ["DF1430", "7AC331", "B055E9", "1913F7", "2D2DA6", "0F9E4E", "DDE088", "209C5A", "E2B400", "6878AA", "34DE81", "62F1E6", "D9953F", "2A75A7", "5D3F46", "132A13", "81D5FE", "CF5C37"]

In [219]:
len(all_lights[0:10])

NameError: name 'all_lights' is not defined

In [220]:
import json

In [221]:
import time

In [222]:
for i in range(len(bri)):
    post(i,'lights', {"deviceid":all_lights_uid[0:10]})
    time.sleep(1)
    post(i,'lights', {"deviceid":all_lights_uid[10:]})
    time.sleep(1)
    

### PARTY MODE

In [66]:
for i in range(len(bri)):
    put(i, 'groups/0/action', 
        {"on": True,
         "hue": 2000,
        })
    time.sleep(1)

In [ ]:
class Controler(object):
    def __init__(self):
        self.targets = {}
        

In [165]:
import pandas as pd

In [166]:
all_lamps = pd.DataFrame(df_lamps)

In [167]:
uid = all_lamps.groupby('uniqueid')

In [168]:
df2 = pd.DataFrame(uid.apply(lambda x: any(x['reachable'])))
df2['bridge'] = uid.apply(lambda x: [x['bridge']])
df2['names'] = uid.apply(lambda x: [x['name']])


In [169]:
df2

,0,bridge,names
uniqueid,,,
00:17:88:01:00:b1:f8:dc-0b,True,[[0]],[[Hue color lamp 17]]
00:17:88:01:00:bf:3a:24-0b,True,[[0]],[[Hue color lamp 16]]
00:17:88:01:00:cf:a5:dc-0b,False,[[0]],[[Hue lightstrip 1]]
00:17:88:01:00:d9:bd:12-0b,True,[[0]],[[P17]]
00:17:88:01:00:d9:c9:cf-0b,False,[[0]],[[P1]]
00:17:88:01:00:d9:ca:66-0b,True,[[0]],[[P19]]
00:17:88:01:00:d9:cb:35-0b,True,[[0]],[[P10]]
00:17:88:01:00:d9:cb:67-0b,True,[[0]],[[P3]]
00:17:88:01:00:d9:cc:a3-0b,True,[[0]],[[Hue color lamp 3]]
